In [ ]:
!pip install optuna
!pip install catboost

In [ ]:
pip install -U imbalanced-learn

In [ ]:
pip install -U torch

In [23]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import catboost as cb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [41]:
# 레이블 인코딩할 칼럼들
cat_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "customer_idx",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "category",
    "product_count",
    "timeline_count",
    "idit_all",
    "lead_owner",
    "bant_submit_count",
    "com_reg_count",
    "idx_count",
    "lead_count",
    "enterprise_count",
    "enterprise_weight"
]

def index_processing(context_df, train, test, column_name):
    idx = {v:k for k,v in enumerate(context_df[column_name].unique())}
    train[column_name] = train[column_name].map(idx)
    test[column_name] = test[column_name].map(idx)
    # train.loc[:, column_name] = train[column_name].map(idx)
    # test.loc[:, column_name] = test[column_name].map(idx)
    return idx

def process_context_data(train_df, test_df):
    context_df = pd.concat([train_df[cat_columns], test_df[cat_columns]]).reset_index(drop=True)
    idx = {}
    for col in cat_columns:
        idx_name = index_processing(context_df, train_df, test_df, col)
        idx[col+'2idx'] = idx_name
    return idx, train_df, test_df

def context_data_load():
    ######################## DATA LOAD
    train = pd.read_csv('train_final.csv', low_memory=False)
    test = pd.read_csv('submission_final.csv')

    idx, context_train, context_test = process_context_data(train, test)
    field_dims = np.array([len(toidx) for toidx in idx], dtype=np.int32)

    data = {
            'train':context_train.fillna(0),
            'test':context_test.fillna(0),
            'field_dims':field_dims,
            'cat_columns' : cat_columns,
            }


    return data

def context_data_split(data):
    # SMOTE를 사용하여 데이터 오버샘플링
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(data['train'].drop(['is_converted'], axis=1), data['train']['is_converted'])

    # 샘플링된 데이터를 다시 훈련 데이터와 테스트 데이터로 분할
    X_train, X_valid, y_train, y_valid = train_test_split(X_resampled, 
                                                      y_resampled, 
                                                      test_size=0.2, 
                                                      random_state=42, 
                                                      stratify=y_resampled)

    y_train = y_train.astype(np.int32) ; y_valid = y_valid.astype(np.int32)
    data['X_train'], data['X_valid'], data['y_train'], data['y_valid'], data['X_resampled'], data['y_resampled'] = X_train, X_valid, y_train, y_valid, X_resampled, y_resampled
    
    return data


In [42]:
data = context_data_load()
data = context_data_split(data)

In [ ]:
# CatBoost 모델 훈련
train_pool = Pool(data['X_train'], label=data['y_train'], cat_features=data['cat_columns'])
valid_pool = Pool(data['X_valid'], label=data['y_valid'], cat_features=data['cat_columns'])

In [ ]:
def objective_catboost(trial):
    param = {
        "random_state": 42,
        "iterations": 
        'early_stopping_rounds': 20,
        'loss_function': 'Logloss',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.00),
        "n_estimators": trial.suggest_int("n_estimators", 500, 3000),
        "max_depth": trial.suggest_int("max_depth", 1, 8),
        'random_strength': trial.suggest_int('random_strength', 0, 50),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "max_bin": trial.suggest_int("max_bin", 150, 300),
    }

    model = cb.CatBoostClassifier(**param, verbose=0)
    model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

    # 검증 데이터셋에 대한 예측 및 정확도 계산
    pred = model.predict(data['X_valid'])
    return f1_score(y_val, y_pred, labels=[True, False])

In [ ]:
# Optuna 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective_catboost, n_trials=2)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
model = cb.CatBoostClassifier(**trial.params, verbose=False)

model.fit(train_pool)

## K-fold

In [43]:
x_train, x_val, y_train, y_val = data['X_train'], data['X_valid'], data['y_train'], data['y_valid']

In [44]:
from sklearn.model_selection import StratifiedKFold
fold_num = 5
skf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(data['X_resampled'], data['y_resampled']):
    folds.append((train_idx,valid_idx))

In [51]:
def objective(trial):
    param = {
        "random_state":42,
        "objective": "Logloss",
        "cat_features" : data['cat_columns'],
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.5),
        'bagging_temperature' :trial.suggest_float('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    }

    model = cb.CatBoostClassifier(**param, devices = '0')

    model.fit(
        x_train,
        y_train.astype(int),
        eval_set=[(x_val, y_val)],
        early_stopping_rounds = 50,
        verbose=100
    )

    # 검증 데이터셋에 대한 예측 및 정확도 계산
    y_pred = model.predict(x_val)
    return f1_score(y_val.astype(int), y_pred.astype(int), labels=[True, False])

for fold in range(0,fold_num):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = data['X_resampled'].iloc[train_idx]
    X_valid = data['X_resampled'].iloc[valid_idx]
    y_train = data['y_resampled'].iloc[train_idx]
    y_valid = data['y_resampled'].iloc[valid_idx]

    
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(
        study_name = 'cat_parameter_optuna',
        direction = 'maximize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=10)

    model = cb.CatBoostClassifier(**study.best_params,
                              devices = '0', random_state = 42, objective = 'Logloss', 
                              cat_features = data['cat_columns'])
    model.fit(X_train, y_train.astype(int), early_stopping_rounds = 50)
                
    pred = model.predict(data['test'].drop(["is_converted", "id"], axis=1))
    data['test'][f'pred_{fold}'] = pred
    print(f'================================================================================\n\n')

[I 2024-02-21 19:21:46,724] A new study created in memory with name: cat_parameter_optuna


====================================1============================================
0:	learn: 0.6931459	test: 0.6931218	best: 0.6931218 (0)	total: 510ms	remaining: 1h 4m 27s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6931218455
bestIteration = 0

Shrink model to first 1 iterations.


[I 2024-02-21 19:22:41,734] Trial 0 finished with value: 0.6005559189012426 and parameters: {'learning_rate': 0.19352465823520762, 'bagging_temperature': 95.07192349792751, 'n_estimators': 7588, 'max_depth': 11, 'random_strength': 15, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.6005559189012426.


0:	learn: 0.6931470	test: 0.6931445	best: 0.6931445 (0)	total: 143ms	remaining: 20m 16s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6931028401
bestIteration = 13

Shrink model to first 14 iterations.


[I 2024-02-21 19:23:10,274] Trial 1 finished with value: 0.5991952414275717 and parameters: {'learning_rate': 0.020086402204943198, 'bagging_temperature': 96.99128611767782, 'n_estimators': 8492, 'max_depth': 6, 'random_strength': 18, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.6005559189012426.


...

[I 2024-02-21 21:27:41,941] Trial 8 finished with value: 0.509617581212592 and parameters: {'learning_rate': 0.4329206786790408, 'bagging_temperature': 62.33357970148751, 'n_estimators': 3978, 'max_depth': 4, 'random_strength': 31, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.6005559189012426.


0:	learn: 0.6931448	test: 0.6931272	best: 0.6931272 (0)	total: 157ms	remaining: 20m 30s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6931271507
bestIteration = 0

Shrink model to first 1 iterations.


[I 2024-02-21 21:28:28,022] Trial 9 finished with value: 0.6005559189012426 and parameters: {'learning_rate': 0.06860118050976784, 'bagging_temperature': 71.32734627442727, 'n_estimators': 7847, 'max_depth': 11, 'random_strength': 77, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 0 with value: 0.6005559189012426.


0:	learn: 0.5294854	total: 400ms	remaining: 50m 31s
1:	learn: 0.3735725	total: 2.4s	remaining: 2h 31m 30s
2:	learn: 0.2420443	total: 4.17s	remaining: 2h 55m 52s
3:	learn: 0.1789102	total: 5.86s	remaining: 3h 5m 5s
4:	learn: 0.1616202	total: 6.23s	remaining: 2h 37m 22s
5:	learn: 0.1406784	total: 8.39s	remaining: 2h 56m 49s
6:	learn: 0.1204955	total: 9.03s	remaining: 2h 42m 55s
7:	learn: 0.1005215	total: 11.2s	remaining: 2h 56m 49s
8:	learn: 0.0923033	total: 13.3s	remaining: 3h 6m 14s
9:	learn: 0.0901101	total: 14.1s	remaining: 2h 57m 37s
10:	learn: 0.0895053	total: 14.6s	remaining: 2h 47m 38s
11:	learn: 0.0848521	total: 16.6s	remaining: 2h 55m 5s
12:	learn: 0.0815776	total: 18.9s	remaining: 3h 3m 55s
13:	learn: 0.0805783	total: 21s	remaining: 3h 8m 57s
14:	learn: 0.0805702	total: 21s	remaining: 2h 57m 5s
15:	learn: 0.0789279	total: 23.2s	remaining: 3h 3m 16s
16:	learn: 0.0778278	total: 24.2s	remaining: 2h 59m 18s
17:	learn: 0.0746797	total: 25.2s	remaining: 2h 56m 58s
18:	learn: 0.07366

Training has stopped (degenerate solution on iteration 176, probably too small l2-regularization, try to increase it)


In [52]:
data['test']

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,...,idx_count,lead_log,lead_count,enterprise_count,enterprise_weight,pred_0,pred_1,pred_2,pred_3,pred_4
0,19844,0.00,5,1,0.073248,50,0,0,53.0,4,...,0,1.098612,1,1,0,1,1,1,0,1
1,9738,0.25,8,2,0.000000,13019,0,1,0.0,7,...,0,3.135494,1,0,0,1,0,1,1,1
2,8491,1.00,6,1,0.000000,35112,1,1,0.0,11,...,0,4.927254,0,0,0,0,0,0,0,0
3,19895,0.50,8,1,0.118644,35113,4,0,0.0,6,...,1,1.098612,1,0,0,0,0,0,0,0
4,10465,1.00,5,1,0.074949,35114,0,0,2.0,2,...,0,4.672829,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,5,0,0.000000,9124,4,0,10.0,6,...,1,1.945910,1,1,0,0,0,0,0,0
5267,7979,0.25,8,2,0.000000,50,4,0,0.0,7,...,0,5.056246,0,0,0,1,1,1,1,1
5268,12887,0.75,5,0,0.000000,38733,1,0,0.0,3,...,1,3.663562,0,0,0,0,0,0,0,0
5269,17530,0.00,39,2,0.000000,38734,0,1,0.0,7,...,1,1.098612,1,0,0,0,1,0,0,0


In [58]:
fold = 1
data['test'][f'pred_{fold}']

0       1
1       0
2       0
3       0
4       0
       ..
5266    0
5267    1
5268    0
5269    1
5270    1
Name: pred_1, Length: 5271, dtype: int64

In [65]:
# 각 array의 i번째 값들을 추출하여 리스트 생성
df_test = data['test']
df_test['is_converted'] = df_test['pred_0'] + df_test['pred_1'] + df_test['pred_2'] + df_test['pred_3'] + df_test['pred_4']
print(df_test['is_converted'].value_counts())
df_test['is_converted'] = df_test['is_converted'].apply(lambda x: True if x > 0 else False)
test_pred = df_test['is_converted']
df_test[['pred_0', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'is_converted']]

0    3715
1     476
5     351
2     273
4     241
3     215
Name: is_converted, dtype: int64


,pred_0,pred_1,pred_2,pred_3,pred_4,is_converted
0,1,1,1,0,1,True
1,1,0,1,1,1,True
2,0,0,0,0,0,False
3,0,0,0,0,0,False
4,0,0,0,0,0,False
...,...,...,...,...,...,...
5266,0,0,0,0,0,False
5267,1,1,1,1,1,True
5268,0,0,0,0,0,False
5269,0,1,0,0,0,True


In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
# 테스트 데이터로 예측
pred = model.predict(data['X_valid'])
get_clf_eval(data['y_valid'], pred)

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
test_pred = model.predict(data['test'].drop(["is_converted", "id"], axis=1))

In [ ]:
test_pred = [val == 1 for val in test_pred]

In [ ]:
sum(test_pred) # True로 예측된 개수

### 제출 파일 작성

In [67]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

df_sub

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,True
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,True
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,True


In [68]:
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)